In [3]:
from math import log
from binascii import hexlify

a = b'shit'
print(a)
print(hexlify(a))

b'shit'
b'73686974'


In [5]:
int(b'shit')

ValueError: invalid literal for int() with base 10: b'shit'

In [13]:
from random import randrange, random
from collections import namedtuple
from math import log, gcd
from binascii import hexlify, unhexlify

def is_prime(n, k=30):
    if n <= 3:
        return n == 2 or n == 3
    neg_one = n - 1

    s, d = 0, neg_one
    while not d & 1:
        s, d = s+1, d>>1
    assert 2 ** s * d == neg_one and d & 1

    for i in range(k):
        a = randrange(2, neg_one)
        x = pow(a, d, n)
        if x in range(1, neg_one):
            continue
        for r in range(1, s):
            x = x ** 2 % n
            if x == 1:
                return False
            if x == neg_one:
                break
        else:
            return False
    return True

def randprime(N=10**8):
    p = 1
    while not is_prime(p):
        p = randrange(N)
    return p

def multinv(modulus, value):
    x, lastx = 0, 1
    a, b = modulus, value
    while b:
        a, q, b = b, a // b, a % b
        x, lastx = lastx - q * x, x
    result = (1 - lastx * modulus) // value
    if result < 0:
        result += modulus
    assert 0 <= result < modulus and value * result % modulus == 1
    return result

KeyPair = namedtuple('KeyPair', 'public private')
Key = namedtuple('Key', 'exponent modulus')

def keygen(N, public=None):
    prime1 = randprime(N)
    prime2 = randprime(N)
    composite = prime1 * prime2
    totient = (prime1 - 1) * (prime2 - 1)
    if public is None:
        while True:
            private = randrange(totient)
            if gcd(private, totient) == 1:
                break
        public = multinv(totient, private)
    else:
        private = multinv(totient, public)
    assert public * private % totient == gcd(public, totient) == gcd(private, totient) == 1
    assert pow(pow(1234567, public, composite), private, composite) == 1234567
    return KeyPair(Key(public, composite), Key(private, composite))

def signature(msg, privkey):
    f=open('signedfile','w')
    coded = pow(int(msg), *privkey)% privkey[1]
    print("Blinded Signed Message "+str(coded))
    f.write(str(coded))

def blindingfactor(N):
    b=random()*(N-1)
    r=int(b)
    while gcd(r,N)!=1:
        r=r+1
    return r

def blind(msg,pubkey):
    f=open('blindmsg','w')
    r=blindingfactor(pubkey[1])
    m=int(msg)
    blindmsg=(pow(r,*pubkey)*m)% pubkey[1]
    print("Blinded Message "+str(blindmsg))
    f.write(str(blindmsg))
    return r


def unblind(msg,r,pubkey):
	f=open('unblindsigned','w')
	bsm=int(msg)
	ubsm=(bsm*multinv(pubkey[1],r))% pubkey[1]
	print("Unblinded Signed Message "+str(ubsm))
	f.write(str(ubsm))


def verefy(msg,r,pubkey):
    print("Message After Verification "+str(pow(int(msg),*pubkey)%pubkey[1]))

In [14]:
f=open('msg')
pubkey, privkey = keygen(2 ** 128)
msg=f.read()
msg=msg.rstrip()
print("Original Message "+str(msg))
r=blind(msg,pubkey)

#Alice receives the blind message and signs it
bf=open('blindmsg')
m=bf.read()
signature(m, privkey)

#Bob recieves the signed message and unblinds it
h=open('signedfile')
signedmsg=h.read()
unblind(signedmsg,r,pubkey)
    
#verifier verefis the message
i=open('unblindsigned')
ubsignedmsg=i.read()
verefy(ubsignedmsg,r,pubkey)

AssertionError: 

In [18]:
s = 'shit'
s[:-2]

'sh'

In [19]:
import uuid
u = uuid.uuid4()
u.int

196086627545314060731611037047031899295

In [31]:
from Crypto import Random
from Crypto.Random import random, get_random_bytes
from Crypto.PublicKey import ElGamal
from Crypto.Util.number import GCD, long_to_bytes
from Crypto.Hash import SHA
import base64


def get_private_key():
    with open('private.el') as f:
        return ElGamal.construct(eval(f.read()))
    
    
message = "Hello"
# key = ElGamal.generate(1024, get_random_bytes)
key = get_private_key()
# h = SHA.new(message.encode()).digest()
h = message.encode()
while 1:
    k = random.StrongRandom().randint(1, key.p-1)
    if GCD(k, key.p-1) == 1: 
        break
sig = key.sign(h, k)

print(base64.encodebytes(long_to_bytes(sig[0])))

if key.verify(h, sig):
    print("OK")
else:
    print("Incorrect signature")


OK


In [32]:
from Crypto import Random
from Crypto.Random import random, get_random_bytes
from Crypto.PublicKey import RSA
from Crypto.Util.number import GCD
from Crypto.Hash import SHA
from uuid import uuid4
import json


def get_private_key():
    with open('private.key') as f:
        return RSA.importKey(f.read())
    
    
def randomize(message):
    d = {
        'message': message,
        'randomize': uuid4().int,
    }
    return json.dumps(d).encode()
    
    
    
message = "Hello"
# key = ElGamal.generate(1024, get_random_bytes)
key = get_private_key()
# h = SHA.new(message.encode()).digest()
h = randomize(message)

blind_factor = random.randrange(key.n >> 10, key.n)
blinded_message = key.blind(h, blind_factor)
    
sig = key.sign(blinded_message, 0)
# print(type(sig))
unblinded_sig = key.unblind(sig[0], blind_factor)

if key.verify(h, (unblinded_sig,)):
    print("OK")
else:
    print("Incorrect signature")

OK


In [20]:
from Crypto import Random
from Crypto.Random import random, get_random_bytes
from Crypto.PublicKey import RSA
from Crypto.Util.number import GCD, long_to_bytes, bytes_to_long
from Crypto.Hash import SHA
from uuid import uuid4
import json
import base64
from binascii import hexlify


def get_private_key():
    with open('private.key') as f:
        return RSA.importKey(f.read())
    
    
def randomize(message):
    d = {
        'message': message,
        'randomize': uuid4().int,
    }
    # return json.dumps(d).encode()
    return message.encode()


message = "Hello"
# key = ElGamal.generate(1024, get_random_bytes)
key = get_private_key()
# h = SHA.new(message.encode()).digest()
h = randomize(message)

sig = key.sign(h, 0)
# print(type(sig))
b64sig = base64.encodebytes(long_to_bytes(sig[0]))
print(b64sig.decode())
print(hexlify(long_to_bytes(sig[0])))
print(list(bytearray(long_to_bytes(sig[0]))))


if key.verify(h, (bytes_to_long(base64.decodebytes(b64sig)), )):
    print("OK")
else:
    print("Incorrect signature")

vSHqKe9M9URC5ECLYNx4eYIjHZF44hIP1/+cQdyRNjhKCpkqjdcqGYMfIzQipAPoGD/gOv+BPUfa
WDXKlv0WMUtj+b6lbuneT1MlaNCz3daDxTt/9iKBEDK/FwHTSkKVL/86kl+phe27Qi2bjWL6uclh
30yFnV2SYt30PSCpUIg=

b'bd21ea29ef4cf54442e4408b60dc787982231d9178e2120fd7ff9c41dc9136384a0a992a8dd72a19831f233422a403e8183fe03aff813d47da5835ca96fd16314b63f9bea56ee9de4f532568d0b3ddd683c53b7ff622811032bf1701d34a42952fff3a925fa985edbb422d9b8d62fab9c961df4c859d5d9262ddf43d20a95088'
[189, 33, 234, 41, 239, 76, 245, 68, 66, 228, 64, 139, 96, 220, 120, 121, 130, 35, 29, 145, 120, 226, 18, 15, 215, 255, 156, 65, 220, 145, 54, 56, 74, 10, 153, 42, 141, 215, 42, 25, 131, 31, 35, 52, 34, 164, 3, 232, 24, 63, 224, 58, 255, 129, 61, 71, 218, 88, 53, 202, 150, 253, 22, 49, 75, 99, 249, 190, 165, 110, 233, 222, 79, 83, 37, 104, 208, 179, 221, 214, 131, 197, 59, 127, 246, 34, 129, 16, 50, 191, 23, 1, 211, 74, 66, 149, 47, 255, 58, 146, 95, 169, 133, 237, 187, 66, 45, 155, 141, 98, 250, 185, 201, 97, 223, 76, 133, 157, 93, 146, 98, 221, 244, 61, 32

In [38]:
base64.decodebytes(b'YdLFyTl2ozdl3VeQClTFOPqAbO61xlQiWjgvyIlvgvMQTzni8D2B0wd6hyEl2vinVHBrqeFz6p2z\nl5WBgaZSrEvs7DgIajCF+hC9krpr1/7qK49Fm2Lmlncus6LjWYjrwTtryMlL0PLWynaho2XRSNfI\nYjsXXDl4vs/jwiMc4h0=\n')

b'a\xd2\xc5\xc99v\xa37e\xddW\x90\nT\xc58\xfa\x80l\xee\xb5\xc6T"Z8/\xc8\x89o\x82\xf3\x10O9\xe2\xf0=\x81\xd3\x07z\x87!%\xda\xf8\xa7Tpk\xa9\xe1s\xea\x9d\xb3\x97\x95\x81\x81\xa6R\xacK\xec\xec8\x08j0\x85\xfa\x10\xbd\x92\xbak\xd7\xfe\xea+\x8fE\x9bb\xe6\x96w.\xb3\xa2\xe3Y\x88\xeb\xc1;k\xc8\xc9K\xd0\xf2\xd6\xcav\xa1\xa3e\xd1H\xd7\xc8b;\x17\\9x\xbe\xcf\xe3\xc2#\x1c\xe2\x1d'

In [7]:
print('shit')

shit


In [4]:
key = ElGamal.generate(1024, get_random_bytes)
print((key.p, key.g, key.y, key.x))

(163792713254893047022994698854544265071948819482106928644010202798045194943479627347801233937528671402471190013660346518801300122146170729950045208162060879362651176589002151293664199588192683288803931075040500177449692875451689901629395357714644063210211503329684453490207330834795063586060945105780639089819, 4013491238258108162078662334995005609192386852712287258203437809010297339754002847964808141580709789416167105015376355414640315978166382748716849914337931473670341639358495454243598807017289522531636803149452604569441458306555511925943197573181109382655903555710070037850462205467022402415740411887931335684, 28770613988250627996402463092133306777741810393388608485626127679322819715665671949020722671255614494347990314924846974474095755731486360602527972935202153874917882341549732688684879523541423280966593987076347115322486018239647672254673744401746091755468705746199732187907227269405952293972376915438386126455, 703577917475713551679973374744232907545807507574864460270381993188739